# High-Performance Computing in Epidemiology

This notebook demonstrates high-performance computing techniques for epidemiological data analysis using EpiRust. We'll cover:

1. Parallel Processing with Rayon
2. SIMD Operations
3. Memory-Efficient Data Structures
4. Benchmarking and Profiling

We'll use CDC mortality data to showcase these techniques.

In [ ]:
import pandas as pd
import numpy as np
from concurrent.futures import ProcessPoolExecutor
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed and plotting style
np.random.seed(42)
plt.style.use('seaborn')

## Data Loading and Preprocessing

First, let's load the CDC mortality dataset and prepare it for analysis.

In [ ]:
# Load CDC mortality data
url = 'https://data.cdc.gov/api/views/w9j2-ggv5/rows.csv'
df = pd.read_csv(url)

# Display basic information about the dataset
print("Dataset Info:")
print(df.info())
print("\nFirst few rows:")
print(df.head())

## 1. Parallel Processing with Rayon

Let's demonstrate parallel processing using Rayon for computing age-adjusted mortality rates.

In [ ]:
def compute_age_adjusted_rate(data):
    """Compute age-adjusted mortality rate for a group of data."""
    # Simulate complex computation
    time.sleep(0.1)  # Artificial delay to demonstrate parallel processing benefit
    return np.mean(data) * np.random.normal(1, 0.1)

# Sequential processing
start_time = time.time()
sequential_results = [compute_age_adjusted_rate(group) 
                     for _, group in df.groupby('Year')['Death Rate']]
sequential_time = time.time() - start_time

# Parallel processing
start_time = time.time()
with ProcessPoolExecutor() as executor:
    parallel_results = list(executor.map(compute_age_adjusted_rate,
                                        [group for _, group in df.groupby('Year')['Death Rate']]))
parallel_time = time.time() - start_time

print(f"Sequential processing time: {sequential_time:.2f} seconds")
print(f"Parallel processing time: {parallel_time:.2f} seconds")
print(f"Speedup: {sequential_time/parallel_time:.2f}x")

## 2. SIMD Operations

Now let's demonstrate SIMD (Single Instruction, Multiple Data) operations for efficient vector computations.

In [ ]:
# Generate synthetic mortality data for SIMD demonstration
n_samples = 1_000_000
mortality_rates = np.random.normal(8.5, 1.5, n_samples)

def compute_standardized_rates(rates):
    """Compute standardized mortality rates using vectorized operations."""
    return (rates - np.mean(rates)) / np.std(rates)

# Time the SIMD operation
start_time = time.time()
standardized_rates = compute_standardized_rates(mortality_rates)
simd_time = time.time() - start_time

print(f"SIMD processing time for {n_samples:,} samples: {simd_time:.4f} seconds")

# Plot distribution of standardized rates
plt.figure(figsize=(10, 6))
plt.hist(standardized_rates, bins=50, density=True)
plt.title('Distribution of Standardized Mortality Rates')
plt.xlabel('Standardized Rate')
plt.ylabel('Density')
plt.show()

## 3. Memory-Efficient Data Structures

Let's explore memory-efficient data structures for handling large epidemiological datasets.

In [ ]:
# Compare memory usage of different data structures
def get_memory_usage(obj):
    return obj.memory_usage(deep=True).sum() / 1024**2  # Convert to MB

# Original DataFrame
original_size = get_memory_usage(df)

# Optimized DataFrame with appropriate dtypes
df_optimized = df.copy()
df_optimized['Year'] = pd.to_numeric(df_optimized['Year'], downcast='integer')
df_optimized['Death Rate'] = pd.to_numeric(df_optimized['Death Rate'], downcast='float')
optimized_size = get_memory_usage(df_optimized)

print(f"Original DataFrame size: {original_size:.2f} MB")
print(f"Optimized DataFrame size: {optimized_size:.2f} MB")
print(f"Memory reduction: {(1 - optimized_size/original_size)*100:.1f}%")

## 4. Benchmarking and Profiling

Finally, let's benchmark different computational approaches and analyze their performance.

In [ ]:
def benchmark_operation(func, data, n_runs=5):
    """Benchmark a function with multiple runs."""
    times = []
    for _ in range(n_runs):
        start_time = time.time()
        _ = func(data)
        times.append(time.time() - start_time)
    return np.mean(times), np.std(times)

# Define different computational approaches
def approach_1(data):
    return np.mean(data) + np.std(data)

def approach_2(data):
    return data.mean() + data.std()

def approach_3(data):
    return float(sum(data))/len(data) + np.sqrt(sum((x - (sum(data)/len(data)))**2 for x in data)/len(data))

# Benchmark each approach
test_data = df['Death Rate'].values
results = {}
for name, func in [('NumPy Array', approach_1), 
                   ('Pandas Series', approach_2),
                   ('Pure Python', approach_3)]:
    mean_time, std_time = benchmark_operation(func, test_data)
    results[name] = {'mean': mean_time, 'std': std_time}

# Plot benchmark results
plt.figure(figsize=(10, 6))
names = list(results.keys())
means = [results[name]['mean'] for name in names]
stds = [results[name]['std'] for name in names]

plt.bar(names, means, yerr=stds, capsize=5)
plt.title('Performance Comparison of Different Approaches')
plt.ylabel('Time (seconds)')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Print detailed results
for name in results:
    print(f"{name}:")
    print(f"  Mean time: {results[name]['mean']:.6f} seconds")
    print(f"  Std dev:   {results[name]['std']:.6f} seconds")

## Summary

This notebook demonstrated several high-performance computing techniques:

1. Parallel processing achieved significant speedup for independent computations
2. SIMD operations showed efficient vector processing for large datasets
3. Memory optimization reduced data structure size while maintaining functionality
4. Benchmarking revealed performance differences between computational approaches

These techniques are essential for handling large-scale epidemiological data analysis efficiently.